<a href="https://colab.research.google.com/github/pathdata/ImageAnnotator/blob/master/Image_stitcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import List, Tuple
import os
from glob import glob
from tqdm import tqdm
import numpy as np

def get_files_from_directory(directory_name:str) -> List[str]:
    return glob(directory_name + '/*.tif', recursive=True)

In [ ]:
def stitch_files(directory_name:str, output_file:str=None) -> None:

    file_list = get_files_from_directory(directory_name)

    def get_coords_from_file_name(file_name:str) -> Tuple[int, int]:
        coords = tuple(map(int, file_name.split('[')[1].split(']')[0].split(',')))
        return coords

    min_x = 0
    min_y = 0

    def get_stitch_resolution(file_list:List[str]) -> Tuple[int, int]:
        nonlocal min_x
        nonlocal min_y
        max_x = 0
        max_y = 0

        for file_name in file_list:
            x, y = get_coords_from_file_name(file_name)
            res_x, res_y = (1396, 1860)

            if (min_x == 0) or (x < min_x):
                min_x = x
            if (min_y == 0) or (y < min_y):
                min_y = y
            if x + res_x > max_x:
                max_x = x + res_x
            if y + res_y > max_y:
                max_y = y + res_y

        return max_x - min_x, max_y - min_y

    canvas = zarr.zeros((8, *get_stitch_resolution(file_list)), chunks=(1, 1396, 1860), dtype=np.float32)

    for file_name in tqdm(file_list):
        x, y = get_coords_from_file_name(file_name)
        store = tifffile.imread(file_name, aszarr=True)
        data = zarr.open(store, mode='r').astype(np.float32)
        canvas[:, x - min_x : x + data.shape[1] - min_x, y - min_y: y + data.shape[2] - min_y] = data

    tifffile.imwrite(f'stitched_{directory_name if output_file is None else output_file}.qptiff', canvas)

In [ ]:
stitch_files('data/1968', '1968')